# Project Psyched: A Closer Look Into Reproducibility In Psychological Research

## Data Analysis & Visualization Script: Part 2b - Rounding Errors
This script for data analysis and visualization after data has been scraped from TDM Studio. This part of the project utilizes the full corpus of both #1 and #2.

Author: Yuyang Zhong (2020). This work is licensed under a [Creative Commons BY-NC-SA 4.0 International
License][cc-by].

![CC BY-NC-SA 4.0][cc-by-shield]

[cc-by]: http://creativecommons.org/licenses/by/4.0/
[cc-by-shield]: https://img.shields.io/badge/license-CC--BY--NC--SA%204.0-blue

#### Setup & Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import ast

from scipy import stats

#### Copied over from data cleaning & transformation script

In [2]:
in_path = "../data/raw/"
out_path = "../data/"
corpus1_stat_path = 'corpus1_stat_full.csv'
corpus2_stat_path = 'corpus2_stat_full.csv'

corpus1_stat = pd.read_csv(in_path + corpus1_stat_path, index_col = 0)
corpus2_stat = pd.read_csv(in_path + corpus2_stat_path, index_col = 0)

corpus1_stat = corpus1_stat.drop(['F_stats_SN', 't_scores_SN'], axis=1)
corpus2_stat = corpus2_stat.drop(['F_stats_SN', 't_scores_SN'], axis=1)

stats_all = corpus1_stat.append(corpus2_stat)
stats_all.head()

,F_stats,F_stats_ns,t_scores,t_scores_ns
614337945.xml,[],[],"['t (41) = 4.10, p < .01', 't (41) ...",[]
1647028895.xml,[],[],[],[]
614404963.xml,"['F(1, 6) = 0.89, p = .382', 'F(1, 63) = 17.02...",[],[],[]
614332724.xml,"['F(10, 776) = 6.84, p < .001', 'F(10, 722) ...",[],[],[]
614304222.xml,[],[],[],[]


In [4]:
df_stats = pd.DataFrame(columns=['File', 'Original', 'Type', 'Param1', 'Param2', 'Stat', 'Sign', 'Reported p-value'])

In [5]:
def extract_f(s):
    """
    Takes in a string of reported F statistics with p-value and extract the numeric parameters.
    
    Returns: df1, df2, x, p
    """
    extract = re.findall(
        r'Fs?\s*\(\s*(\d+)\s*\,\s*(\d+)\s*\)\s*[\<|\>|\=]\s*(\d*\.?\d*)\s*\,\s*p\s*([\<|\>|\=])\s*(\d*\.\d+)',
        s)[0]    
    
    df1 = float(extract[0])
    df2 = float(extract[1])
    
    x = float(extract[2])
    ineq = extract[3]
    p = float(extract[4])
    
    return df1, df2, x, ineq, p

def extract_t(s):
    """
    Takes in a string of reported t score with p-value and extract the numeric parameters.
    
    Returns: df, x, p
    """
    
    s = re.sub(r'−\s*', "-", s)
    extract = re.findall(
        r't\s*\((\s*\d*\s*,)?\s*(\d+)\s*\)\s*[\<|\>|\=]\s*([\−|\-]?\s*\d*\.?\d*)\s*,\s*p\s*([\<|\>|\=])\s*(\d?\.\d+)',
        s)[0]
    
    df = float(extract[1])
    x = float(extract[2].replace(" ", ""))
    ineq = extract[3]
    p = float(extract[4])
    
    return df, x, ineq, p

In [6]:
def extract_f_compute_add(test_type, index, s):
    ex = extract_f(s)
    return {'File': index, 
            'Original': s, 
            'Type': test_type,
            'Param1': ex[0], 
            'Param2': ex[1], 
            'Stat': ex[2], 
            'Sign': ex[3],
            'Reported p-value': ex[4]}

def extract_t_compute_add(test_type, index, s):
    ex = extract_t(s)
    return {'File': index, 
            'Original': s, 
            'Type': test_type, 
            'Param1': ex[0], 
            'Param2': None,
            'Stat': ex[1],
            'Sign': ex[2],
            'Reported p-value': ex[3]}

In [7]:
for col in stats_all.columns:
    stats_all[col] = stats_all[col].apply(ast.literal_eval)

In [10]:
count = 0

for index, row in stats_all.iterrows():
    
    if len(row['F_stats']) > 0:
        for s in row['F_stats']:
            if count % 1000 == 0:
                print(count, 'p-values captured')
            
            try:
                extract = extract_f_compute_add('f', index, s)
            except:
                pass
            else:
                df_stats = df_stats.append(extract, ignore_index=True)
            count += 1
    
    if len(row['t_scores']) > 0:
        for s in row['t_scores']:
            if count % 1000 == 0:
                print(count, 'p-values captured')
        
            try:
                extract = extract_t_compute_add('t', index, s)
            except:
                pass
            else:
                df_stats = df_stats.append(extract, ignore_index=True)
            count += 1
        

0 p-values captured
1000 p-values captured
2000 p-values captured
3000 p-values captured
4000 p-values captured
5000 p-values captured
6000 p-values captured
7000 p-values captured
8000 p-values captured
9000 p-values captured
10000 p-values captured
11000 p-values captured
12000 p-values captured
13000 p-values captured
14000 p-values captured
15000 p-values captured
16000 p-values captured
17000 p-values captured
18000 p-values captured
19000 p-values captured
20000 p-values captured
21000 p-values captured
22000 p-values captured
23000 p-values captured
24000 p-values captured
25000 p-values captured
26000 p-values captured
27000 p-values captured
28000 p-values captured
29000 p-values captured
30000 p-values captured
31000 p-values captured
32000 p-values captured
33000 p-values captured
34000 p-values captured
35000 p-values captured
36000 p-values captured
37000 p-values captured
38000 p-values captured
39000 p-values captured
40000 p-values captured
41000 p-values captured
42000

KeyboardInterrupt: 

In [11]:
df_stats.to_csv(out_path + 'stats_all_w_param.csv')

#### Recalcuating p-values, accounting for rounding errors

In [13]:
df_stats

,File,Original,Type,Param1,Param2,Stat,Sign,Reported p-value
0,614337945.xml,"t (41) = 4.10, p < .01",t,41.0,None,4.10,<,0.0100
1,614337945.xml,"t (41) = −3.56, p < .01",t,41.0,None,-3.56,<,0.0100
2,614337945.xml,"t (41) = 8.21, p < .01",t,41.0,None,8.21,<,0.0100
3,614337945.xml,"t (41) = 4.82, p < .01",t,41.0,None,4.82,<,0.0100
4,614337945.xml,"t (41) = −2.57, p < .01",t,41.0,None,-2.57,<,0.0100
...,...,...,...,...,...,...,...,...
197786,614478460.xml,"t(99) = 42.24, p < .0001",t,99.0,None,42.24,<,0.0001
197787,614338318.xml,"F(9, 214) = 3.84, p < .0001",f,9.0,214,3.84,<,0.0001
197788,614338318.xml,"F(9, 240) = 0.94, p = .50",f,9.0,240,0.94,=,0.5000
197789,614338318.xml,"F(3, 236) = 6.10, p = .0005",f,3.0,236,6.10,=,0.0005


In [15]:
def recalculate(row):
    if row['Type'] == 't':
        return 1-stats.t.cdf(row['Stat'], row['Param1'])
    else:
        return 1-stats.f.cdf(row['Stat'], row['Param1'], row['Param2'])

In [16]:
df_stats['Recalculated p-value'] = df_stats.apply(lambda x: recalculate(x), axis=1)

Taking any rows that has a reported p-value around $[0.045, 0.05]$

In [44]:
close_df = df_stats[(df_stats['Reported p-value'] - 0.05 >= -0.005) & (df_stats['Reported p-value'] - 0.05 <= 0)]
close_df.shape

(30066, 9)

In [34]:
def recalculate_round_up(row):
    if row['Type'] == 't':
        return 1-stats.t.cdf(row['Stat']+0.1, row['Param1'])
    else:
        return 1-stats.f.cdf(row['Stat']+0.1, row['Param1'], row['Param2'])
    
def recalculate_round_down(row):
    if row['Type'] == 't':
        return 1-stats.t.cdf(row['Stat']-0.1, row['Param1'])
    else:
        return 1-stats.f.cdf(row['Stat']-0.1, row['Param1'], row['Param2'])

In [35]:
close_df['Stat+0.1'] = close_df.apply(lambda x: recalculate_round_up(x), axis=1)
close_df['Stat-0.1'] = close_df.apply(lambda x: recalculate_round_down(x), axis=1)

<ipython-input-35-cedd26c7be4c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_df['Stat+0.1'] = close_df.apply(lambda x: recalculate_round_up(x), axis=1)
<ipython-input-35-cedd26c7be4c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_df['Stat-0.1'] = close_df.apply(lambda x: recalculate_round_down(x), axis=1)


In [45]:
close_df[close_df['Type'] == 'f'].shape

(20382, 9)

From those rows with close to 0.05 p-values, reculculated and see which ones are mismatching. Looking at F-test only because t-scores are not yet corrected.

In [43]:
mismatch_df = close_df[(close_df['Recalculated p-value'] >=0.05) & (close_df['Type']=='f')]
mismatch_df.shape

(3085, 11)

In [47]:
mismatch_df[(mismatch_df['Stat+0.1'] < 0.05) | (mismatch_df['Stat-0.1'] < 0.05)]

,File,Original,Type,Param1,Param2,Stat,Sign,Reported p-value,Recalculated p-value,Stat+0.1,Stat-0.1
25,614357009.xml,"F(2, 57) = 3.1, p = .05",f,2.0,57,3.10,=,0.05,0.052723,0.048183,0.057707
351,614317829.xml,"F(1, 48) = 4.02, p = .05",f,1.0,48,4.02,=,0.05,0.050621,0.047942,0.053464
586,614496153.xml,"F(2, 95) = 3.01, p = .05",f,2.0,95,3.01,=,0.05,0.054016,0.049172,0.059348
718,614307900.xml,"F(1, 39) = 4.05, p < .05",f,1.0,39,4.05,<,0.05,0.051114,0.048462,0.053926
784,1753445757.xml,"F(2, 416) = 2.98, p = .05",f,2.0,416,2.98,=,0.05,0.051878,0.047009,0.057255
...,...,...,...,...,...,...,...,...,...,...,...
197100,1685750389.xml,"F(1, 114) = 3.88, p = .05",f,1.0,114,3.88,=,0.05,0.051290,0.048429,0.054333
197244,614332498.xml,"F(4, 212) = 2.37, p = .05",f,4.0,212,2.37,=,0.05,0.053632,0.045759,0.062806
197388,614404428.xml,"F(2, 27) = 3.33, p < .05",f,2.0,27,3.33,<,0.05,0.050977,0.047058,0.055247
197588,614320914.xml,"F(2, 80) = 3.02, p = .05",f,2.0,80,3.02,=,0.05,0.054398,0.049573,0.059705


Also to note that subtracting 0.1 from the statistic did not help the calculation.